<a href="https://colab.research.google.com/github/nshomoud/travel-planner/blob/main/_travel_planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git remote add origin https://github.com/nshomoud/travel-planner.git
!git branch -M main
!git push -u origin main

# Install necessary packages
!pip install flask pandas requests

# Import required libraries
from flask import Flask, request, jsonify
import pandas as pd
import threading
import requests

# Load and preprocess the dataset
file_path = '/content/Travel_details_dataset.csv'
df = pd.read_csv(file_path)
df['Traveler nationality'] = df['Traveler nationality'].str.lower()
df['Destination'] = df['Destination'].str.lower()
df['Accommodation type'] = df['Accommodation type'].str.lower()

app = Flask(__name__)

def recommend_destinations(nationality, min_duration):
    recommendations = df[(df['Traveler nationality'] == nationality) &
                         (df['Duration (days)'] >= min_duration)]
    if not recommendations.empty:
        return recommendations[['Destination', 'Duration (days)']].to_dict(orient='records')
    return []

def recommend_accommodations(destination, accommodation_type):
    accommodations = df[(df['Destination'] == destination) &
                        (df['Accommodation type'] == accommodation_type)]
    if not accommodations.empty:
        return accommodations[['Accommodation type', 'Accommodation cost']].to_dict(orient='records')
    return []

@app.route('/travel_planner', methods=['POST'])
def travel_planner():
    data = request.get_json()
    request_type = data.get('request_type', '').lower()

    if request_type == 'destination':
        nationality = data.get('nationality', '').lower()
        min_duration = data.get('min_duration', 0)
        recommendations = recommend_destinations(nationality, min_duration)
        if recommendations:
            return jsonify({
                "request_type": "destination",
                "nationality": nationality,
                "min_duration": min_duration,
                "recommendations": recommendations
            })
        return jsonify({"message": "No destinations found for your criteria."})

    elif request_type == 'accommodation':
        destination = data.get('destination', '').lower()
        accommodation_type = data.get('accommodation_type', '').lower()
        recommendations = recommend_accommodations(destination, accommodation_type)
        if recommendations:
            return jsonify({
                "request_type": "accommodation",
                "destination": destination,
                "accommodation_type": accommodation_type,
                "recommendations": recommendations
            })
        return jsonify({"message": f"No accommodations of type '{accommodation_type}' found for {destination}."})

    return jsonify({"message": "Invalid request. Please specify 'destination' or 'accommodation' along with appropriate parameters."})

def run_app():
    app.run(port=5000)

# Run Flask app in a separate thread
threading.Thread(target=run_app).start()

# Simulate user queries
import time
time.sleep(3)  # Wait for the server to start

# Function to format and print recommendations
def print_recommendations(response_json):
    if "recommendations" in response_json:
        print("\nRecommendations:")
        for item in response_json["recommendations"]:
            print(f"- {', '.join([f'{k}: {v}' for k, v in item.items()])}")
    else:
        print("\n", response_json.get("message", "No recommendations found."))

# Loop to continue asking for new recommendations
while True:
    # Example user input for destination recommendation
    nationality = input("Enter your nationality: ").lower()
    min_duration = int(input("Enter minimum duration of trip (in days): "))

    data = {
        "request_type": "destination",
        "nationality": nationality,
        "min_duration": min_duration
    }

    response = requests.post('http://127.0.0.1:5000/travel_planner', json=data)
    print("Destination Recommendations:")
    print_recommendations(response.json())

    # Example user input for accommodation recommendation
    destination = input("Enter your destination: ").lower()
    accommodation_type = input("Enter preferred type of accommodation (e.g., hotel, airbnb): ").lower()

    data = {
        "request_type": "accommodation",
        "destination": destination,
        "accommodation_type": accommodation_type
    }

    response = requests.post('http://127.0.0.1:5000/travel_planner', json=data)
    print("Accommodation Recommendations:")
    print_recommendations(response.json())

    # Ask the user if they want another recommendation
    another_recommendation = input("\nWould you like another recommendation? (yes/no): ").lower()
    if another_recommendation != 'yes':
        print("Exiting the program. Goodbye!")
        break


fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Enter your nationality: british
Enter minimum duration of trip (in days): 6


INFO:werkzeug:127.0.0.1 - - [10/Oct/2024 12:21:56] "POST /travel_planner HTTP/1.1" 200 -


Destination Recommendations:

Recommendations:
- Destination: new york, usa, Duration (days): 14.0
- Destination: london, Duration (days): 6.0
- Destination: australia, Duration (days): 13.0
- Destination: mexico, Duration (days): 10.0
- Destination: london, uk, Duration (days): 7.0
- Destination: london, uk, Duration (days): 8.0
- Destination: cape town, Duration (days): 9.0
- Destination: bali, Duration (days): 8.0
- Destination: paris, france, Duration (days): 7.0
- Destination: phuket, thai, Duration (days): 7.0
- Destination: london, Duration (days): 11.0
- Destination: tokyo, japan, Duration (days): 7.0
Enter your destination: london
Enter preferred type of accommodation (e.g., hotel, airbnb): hotel


INFO:werkzeug:127.0.0.1 - - [10/Oct/2024 12:22:05] "POST /travel_planner HTTP/1.1" 200 -


Accommodation Recommendations:

Recommendations:
- Accommodation cost: $1,200 , Accommodation type: hotel
- Accommodation cost: $600 , Accommodation type: hotel
- Accommodation cost: 900, Accommodation type: hotel
